In [1]:

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import cPickle as pickle
import os; import sys; sys.path.append('..')
import gp
import gp.nets as nets

Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled, CuDNN 4007)
/home/d/nolearn/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [3]:
import numpy as np
import mahotas as mh
import h5py

In [9]:
import glob
golds = np.zeros((50,512,512), dtype=np.uint64)

path = '/home/d/data/cylinderNEW/'
for z in range(50,100):
    gold = sorted(glob.glob(os.path.join(path, 'gold', '*'+str(z)+'.png')))
    gold = mh.imread(gold[0])
    gold_single = np.zeros((gold.shape[0], gold.shape[1]), dtype=np.uint64)
    gold_single[:, :] = gold[:,:,0]*256*256 + gold[:,:,1]*256 + gold[:,:,2]
    
    golds[z-50] = gold_single[768:768+512,768:768+512]
#
# create gold h5 for neuroproof
#
all_labels = mh.fullhistogram(golds.astype(np.uint64))
all_labels = np.where(all_labels!=0)

transforms = np.zeros((all_labels[0].shape[0],2), dtype=np.uint64)
transforms[:,0] = all_labels[0]
transforms[:,1] = all_labels[0]
print 'generated transforms.'

with h5py.File('/home/d/fp_data/c_gold_small.h5', 'w') as hf:
    hf.create_dataset('stack', data=golds)
    hf.create_dataset('transforms', data=transforms)

generated transforms.


In [7]:
import glob
rhoanas = np.zeros((50,512,512), dtype=np.uint64)

path = '/home/d/data/cylinderNEW/'
for z in range(50,100):
    rhoana = sorted(glob.glob(os.path.join(path, 'rhoana', '*'+str(z)+'.png')))
    rhoana = mh.imread(rhoana[0])
    rhoana_single = np.zeros((rhoana.shape[0], rhoana.shape[1]), dtype=np.uint64)
    rhoana_single[:, :] = rhoana[:,:,0]*256*256 + rhoana[:,:,1]*256 + rhoana[:,:,2]
    
    rhoanas[z-50] = rhoana_single[768:768+512,768:768+512]
    
#
# create rhoana h5 for neuroproof
#
all_labels = mh.fullhistogram(rhoanas.astype(np.uint64))
all_labels = np.where(all_labels!=0)

transforms = np.zeros((all_labels[0].shape[0],2), dtype=np.uint64)
transforms[:,0] = all_labels[0]
transforms[:,1] = all_labels[0]

with h5py.File('/home/d/fp_data/training/c_rhoana_small.h5', 'w') as hf:
    hf.create_dataset('stack', data=rhoanas)
    hf.create_dataset('transforms', data=transforms)

In [8]:
import glob
import tifffile as tif
probs = np.zeros((50,512,512), dtype=np.uint64)

path = '/home/d/data/cylinderNEW/'
for z in range(50,100):
    prob = sorted(glob.glob(os.path.join(path, 'prob', '*'+str(z)+'.tif')))
    prob = tif.imread(prob[0])
    mask = sorted(glob.glob(os.path.join(path, 'mask', '*'+str(z)+'.png')))
    mask = mh.imread(mask[0]).astype(np.bool)
    masked_prob = prob
    masked_prob = 255 - masked_prob
    masked_prob[mask==0] = 0
    
    probs[z-50] = masked_prob[768:768+512,768:768+512]
    

#
# create prob h5 for neuroproof
#
p1 = probs
# has to be x,y,z
p1 = p1.swapaxes(0,2)
p1 = p1.swapaxes(0,1)

p2 = p1.copy()
p2 = 255-p2
p_2channels = np.zeros((probs.shape[2],probs.shape[1],probs.shape[0],2), dtype=np.float32)
p_2channels[:,:,:,0] = p1 / 255.
p_2channels[:,:,:,1] = p2 / 255.
with h5py.File('/home/d/fp_data/training/c_prob_small.h5', 'w') as hf:
    hf.create_dataset('volume/predictions', data=p_2channels)

In [4]:
sys.path.append('../gp/')